In [ ]:
# !pip install moviepy
# !pip install google-generativeai
# !pip install elevenlabs
# !pip install python-dotenv
# !pip install requests
# !pip install mutagen
!pip install playsound

In [19]:
import os
import time
import requests
import uuid
from requests.auth import HTTPBasicAuth
from elevenlabs.client import ElevenLabs
from dotenv import load_dotenv
from IPython.display import Audio, display
from mutagen.mp3 import MP3

import warnings
warnings.filterwarnings("ignore")

In [20]:
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY")
VOICE_ID = os.getenv("VOICE_ID")
GEMINI_LLM_MODEL = os.getenv("GEMINI_LLM_MODEL")
LOL_LOCKFILE_PATH = os.getenv("LOL_LOCKFILE_PATH")
POLL_INTERVAL = os.getenv("POLL_INTERVAL")

gemini_api_url = f"https://generativelanguage.googleapis.com/v1beta/models/{GEMINI_LLM_MODEL}:generateContent?key={GEMINI_API_KEY}"

In [21]:
def read_lockfile(path=LOL_LOCKFILE_PATH):
    with open(path, "r") as f:
        name, pid, port, password, protocol = f.read().split(":")
    return port, password

port, password = read_lockfile()
lcu_auth = HTTPBasicAuth("riot", password)

In [22]:
def lcu_request(endpoint):
    """Generic GET request to LCU"""
    lcu_url = f"https://127.0.0.1:{port}{endpoint}"
    resp = requests.get(lcu_url, auth=lcu_auth, verify=False)
    return resp.json()

def live_request(endpoint):
    """Generic GET request to Live Client API"""
    url = f"https://127.0.0.1:2999{endpoint}"
    resp = requests.get(url, verify=False)
    return resp.json()

In [23]:
def get_current_summoner():
    return lcu_request("/lol-summoner/v1/current-summoner")

def get_gameflow_phase():
    return lcu_request("/lol-gameflow/v1/gameflow-phase")

def get_champselect_session():
    return lcu_request("/lol-champ-select/v1/session")

def get_active_player():
    return live_request("/liveclientdata/activeplayer")

def get_player_list():
    return live_request("/liveclientdata/playerlist")

def get_game_stats():
    return live_request("/liveclientdata/gamestats")

def get_event_data():
    return live_request("/liveclientdata/eventdata")

ALL_ENDPOINTS = {
    "LCU": {
        "Current Summoner": get_current_summoner,
        "Gameflow Phase": get_gameflow_phase,
        "Champ Select Session": get_champselect_session,
    },
    "Live Client": {
        "Active Player": get_active_player,
        "Player List": get_player_list,
        "Game Stats": get_game_stats,
        "Event Data": get_event_data
    },
}

for category, endpoints in ALL_ENDPOINTS.items():
    print(f"\n==== {category} Endpoints ====")
    for name, func in endpoints.items():
        try:
            print(f"\n--- {name} ---")
            data = func()
            print(data)
        except Exception as e:
            print(f"Failed to fetch {name}: {e}")


==== LCU Endpoints ====

--- Current Summoner ---
{'accountId': 238448945, 'displayName': '', 'gameName': 'CritHappens', 'internalName': '', 'nameChangeFlag': False, 'percentCompleteForNextLevel': 74, 'privacy': 'PUBLIC', 'profileIconId': 6623, 'puuid': '24fd96a4-0a74-5c94-8438-f56d5bfa9ab6', 'rerollPoints': {'currentPoints': 500, 'maxRolls': 2, 'numberOfRolls': 2, 'pointsCostToRoll': 250, 'pointsToReroll': 0}, 'summonerId': 105732387, 'summonerLevel': 138, 'tagLine': '6969', 'unnamed': False, 'xpSinceLastLevel': 2447, 'xpUntilNextLevel': 3264}

--- Gameflow Phase ---


InProgress

--- Champ Select Session ---
{'errorCode': 'RPC_ERROR', 'httpStatus': 404, 'implementationDetails': {}, 'message': 'No active delegate'}

==== Live Client Endpoints ====

--- Active Player ---
{'abilities': {'E': {'abilityLevel': 0, 'displayName': 'Bladecaller', 'id': 'XayahE', 'rawDescription': 'GeneratedTip_Spell_XayahE_Description', 'rawDisplayName': 'GeneratedTip_Spell_XayahE_DisplayName'}, 'Passive': {'displayName': 'Clean Cuts', 'id': 'XayahPassive', 'rawDescription': 'GeneratedTip_Passive_XayahPassive_Description', 'rawDisplayName': 'GeneratedTip_Passive_XayahPassive_DisplayName'}, 'Q': {'abilityLevel': 0, 'displayName': 'Double Daggers', 'id': 'XayahQ', 'rawDescription': 'GeneratedTip_Spell_XayahQ_Description', 'rawDisplayName': 'GeneratedTip_Spell_XayahQ_DisplayName'}, 'R': {'abilityLevel': 0, 'displayName': 'Featherstorm', 'id': 'XayahR', 'rawDescription': 'GeneratedTip_Spell_XayahR_Description', 'rawDisplayName': 'GeneratedTip_Spell_XayahR_DisplayName'}, 'W': {'a

In [24]:
# ======== LLM GENERATION ========
chat_history = []

def get_caption_from_gemini(event_or_context_text):
    """
    Generates a commentary caption for League of Legends game events.

    Args:
        event_or_context_text (str): A string containing relevant game events
                                     from the LCU API.

    Returns:
        str: The generated commentary caption.
    """
    global chat_history

    # Define the system prompt for the commentator persona
    system_prompt = """
        You are a professional League of Legends esports commentator. 
        Your job is to provide an exciting and engaging play-by-play commentary.
        Use a vibrant and energetic tone. Focus on the most important events like kills, objectives taken (Dragons, Barons, Towers), and teamfights.
        Keep your commentary concise and impactful. Do not state that you are an AI model.
        Don't use any special caracters like *
    """

    # Construct the user's prompt with the provided events
    user_prompt = f"The following events just happened in the game:\n{event_or_context_text}\n\nProvide commentary based on these events."
    
    # Add the current user prompt to the chat history
    chat_history.append({"role": "user", "parts": [{"text": user_prompt}]})

    # Prepare the payload for the Gemini API call
    payload = {
        "contents": chat_history,
        "systemInstruction": {
            "parts": [{"text": system_prompt}]
        }
    }    

    # Use exponential backoff for the API call
    max_retries = 3
    for attempt in range(max_retries):
        try:
            # Here you would make the actual network request
            response = requests.post(gemini_api_url, json=payload)
            response.raise_for_status()
            result = response.json()

            candidate = result.get("candidates", [])[0]
            text = candidate["content"]["parts"][0]["text"]
            
            # Add the model's response to the chat history for context
            chat_history.append({"role": "model", "parts": [{"text": text}]})
            
            # Return the generated caption
            return text

        except requests.exceptions.RequestException as e:
            if attempt < max_retries - 1:
                wait_time = 2 ** attempt
                print(f"API call failed. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                print(f"API call failed after {max_retries} attempts: {e}")
                return "Our commentator seems to be having a technical issue. Please stand by."

In [ ]:
if ELEVENLABS_API_KEY and VOICE_ID:
    elevenlabs_client = ElevenLabs(api_key=ELEVENLABS_API_KEY)
else:
    print("Warning: ElevenLabs API key or voice ID not found. Audio generation will be skipped.")

def get_audio_from_elevenlabs(text_to_speak):
    """
    Generates audio, saves it, plays it, and sleeps for its duration.
    """
    audio = elevenlabs_client.text_to_speech.convert(
        text=text_to_speak,
        voice_id=VOICE_ID,
        model_id="eleven_flash_v2",
        output_format="mp3_44100_128",
    )
    
    filename = str(uuid.uuid4())
    save_file_path = f"{filename}.mp3"

    with open(save_file_path, "wb") as f:
        for chunk in audio:
            if chunk:
                f.write(chunk)
    
    # Get the audio duration using mutagen
    try:
        audio_file = MP3(save_file_path)
        audio_duration = audio_file.info.length
    except Exception as e:
        print(f"Error getting audio duration: {e}")
        audio_duration = 0 

    # Play the audio
    #display(Audio(filename=save_file_path, autoplay=True))

    # Sleep for the duration of the audio
    if audio_duration > 0:
        time.sleep(audio_duration)

    os.remove(save_file_path)

In [26]:
class LoLContext:
    """Stores game context for narration"""
    def __init__(self):
        self.seen_event_ids = set()
        self.champ_select_done = False
        self.players_info = []
        self.teams_info = {}
    
    def update_champ_select(self):
        try:
            session = get_champselect_session()
            self.players_info = session.get("myTeam", [])
            self.teams_info = {
                "myTeam": session.get("myTeam", []),
                "theirTeam": session.get("theirTeam", [])
            }
            if self.players_info:
                self.champ_select_done = True
        except:
            pass

    def get_new_events(self):
        events = []
        try:
            all_events = get_event_data().get("Events", [])
            for e in all_events:
                if e["EventID"] not in self.seen_event_ids:
                    self.seen_event_ids.add(e["EventID"])
                    events.append(e)
        except:
            pass
        return events

In [27]:
def event_to_text(event):
    etype = event["EventName"]
    t = int(event.get("EventTime",0))
    mm, ss = divmod(t, 60)

    if etype == "GameStart":
        desc = "The game has started!"
    elif etype == "MinionsSpawning":
        desc = "Minions have spawned."
    elif etype == "ChampionKill":
        desc = f'{event.get("KillerName","?")} killed {event.get("VictimName","?")}'
    elif etype == "TurretKilled":
        desc = f'{event.get("KillerName","?")} destroyed {event.get("TurretKilled","a turret")}'
    elif etype == "DragonKill":
        desc = f'{event.get("KillerName","?")} killed a {event.get("DragonType","dragon")} dragon'
    elif etype == "BaronKill":
        desc = f'{event.get("KillerName","?")} killed Baron Nashor'
    else:
        desc = etype

    return f"[{mm:02d}:{ss:02d}] {desc}"

In [28]:
def main_loop():
    is_first_run = True
    ctx = LoLContext()
    while True:
        # Handle the initial welcome message on the first run
        if is_first_run:
            is_first_run = False
            get_audio_from_elevenlabs("""
                Welcome, ladies and gentlemen, to the ultimate battleground, where legends are made and history is written! 
                I am your host, bringing you the fastest plays, sharpest calls, and spectacular highlights from today s high-stakes tournament. 
                Get ready for insane strategies, jaw-dropping moves, and edge-of-your-seat action as our top contenders prove why they deserve to be called the best in the game. 
                Strap in—because the only thing faster than the gameplay tonight is the excitement in the arena!
            """)
        phase = get_gameflow_phase()
        # 1. Pregame: champ select
        if phase in ["Lobby", "Matchmaking", "ChampSelect"] and not ctx.champ_select_done:
            ctx.update_champ_select()
            if ctx.champ_select_done:
                text = "Champ select is done. Teams and bans are set."
                caption = get_caption_from_gemini(text)
                print(caption)
                get_audio_from_elevenlabs(caption)
        
        # 2. In-game: fetch events
        if phase == "InProgress":
            context = ""
            new_events = ctx.get_new_events()
            for e in new_events:
                text = event_to_text(e)
                context = context + text + "\n"

            caption = get_caption_from_gemini(context)
            print(caption)
            get_audio_from_elevenlabs(caption)
        
        # time.sleep(int(POLL_INTERVAL))

In [29]:
# ======== RUN ========
print("🚀 Starting AI commentator...")
main_loop()

🚀 Starting AI commentator...


KeyboardInterrupt: 